# imorting dependencies

In [1]:
import scipy.io
import numpy as np
from datetime import datetime, timedelta
import cv2
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
from tkinter import filedialog, Label
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from PIL import Image, ImageTk
import tensorflow as tf
import tkinter as tk
import cv2
import numpy as np


# Load and Extract Data from .mat File

In [ ]:
# Load the .mat file
data_path = r"C:\Users\Harsh\Downloads\data_science01\dl_projects\Age_data_work\data\wiki_crop\wiki_crop\wiki.mat"
data = scipy.io.loadmat(data_path)

# Access the 'wiki' structure
wiki = data['wiki']

# Extracting the fields
dob = wiki['dob'][0][0][0]  # Date of birth
photo_taken = wiki['photo_taken'][0][0][0]  # Year photo was taken
full_path = wiki['full_path'][0][0][0]  # Image paths
face_location = wiki['face_location'][0][0][0]  # Face location coordinates


# Convert DOB to Year and Calculate Ages

In [ ]:
# Convert DOB to the year of birth
reference_date = datetime(1, 1, 1)
dob_dates = np.array([reference_date + timedelta(days=int(d)) for d in dob])
dob_years = np.array([date.year for date in dob_dates])

# Calculate ages
ages = photo_taken - dob_years


# Verify Data Extraction

In [ ]:
# Check the length of each array
print("Lengths of the arrays:")
print("DOB:", len(dob))
print("Photo Taken:", len(photo_taken))
print("Full Path:", len(full_path))
print("Face Location:", len(face_location))

# Print example values to ensure correctness
print("\nExample values:")
print("DOB:", dob[:5])
print("DOB Dates:", dob_dates[:5])
print("DOB Years:", dob_years[:5])
print("Photo Taken:", photo_taken[:5])
print("Full Path:", full_path[:5])
print("Face Location:", face_location[:5])
print("Ages:", ages[:5])


# Load Images and Labels

In [ ]:
# Initialize lists for images and labels
images = []
labels = []

# Base path to images
image_base_path = r"C:\Users\Harsh\Downloads\data_science01\dl_projects\Age_data_work\data\wiki_crop\wiki_crop"

# Load images and corresponding ages
for i, path in enumerate(full_path):
    img_path = os.path.join(image_base_path, path[0])
    if os.path.exists(img_path):
        img = cv2.imread(img_path)
        if img is not None:
            images.append(img)
            labels.append(ages[i])
        else:
            print(f"Image not loaded properly: {img_path}")
    else:
        print(f"Image not found: {img_path}")

print(f"Loaded {len(images)} images")


# Splitting Data into Training and Validation Sets

In [ ]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

# Verify the shapes of the arrays
print(f"X_train shape: {len(X_train)}, y_train shape: {len(y_train)}")
print(f"X_val shape: {len(X_val)}, y_val shape: {len(y_val)}")


# preprocessing Images

In [ ]:
def preprocess_images_in_batches(image_list, batch_size=1000, target_size=(128, 128)):
    processed_images = []
    num_images = len(image_list)
    
    for i in range(0, num_images, batch_size):
        batch_images = image_list[i:i+batch_size]
        if not batch_images:
            continue
        processed_batch = []
        for img in batch_images:
            img_resized = cv2.resize(img, target_size)
            img_normalized = img_resized / 255.0  # Normalize to [0, 1] range
            processed_batch.append(img_normalized)
        processed_images.append(np.array(processed_batch, dtype=np.float32))
        print(f"Processed batch {i // batch_size + 1} / {num_images // batch_size + 1}")
    
    return np.concatenate(processed_images, axis=0)

# Preprocess the images
X_train = preprocess_images_in_batches(X_train)
X_val = preprocess_images_in_batches(X_val)

# Convert y_train and y_val to numpy arrays
y_train = np.array(y_train, dtype=np.int32)
y_val = np.array(y_val, dtype=np.int32)


# data augumentation

In [ ]:
# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
datagen.fit(X_train)


# Defining and compiling cnn model

In [ ]:
# Define the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Conv2D(256, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    GlobalAveragePooling2D(),
    Dense(512, activation='relu', kernel_regularizer=l2(0.01)),
    Dropout(0.4),
    Dense(1)
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])


# Training the model

In [ ]:
# Learning rate scheduler
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.0001)

# Train the model with data augmentation
history = model.fit(datagen.flow(X_train, y_train, batch_size=64),
                    epochs=30, validation_data=(X_val, y_val),
                    callbacks=[reduce_lr])


# Evaluating the model

In [ ]:
# Evaluate the model
loss, mae = model.evaluate(X_val, y_val)
print(f"Validation Mean Absolute Error: {mae}")


# Saving the model

In [ ]:
# Save the model
model.save(r"C:\Users\Harsh\Downloads\data_science01\dl_projects\Age_data_work\Trained cnn model")

# GUI

In [6]:
# Load the trained model
model = load_model(r"C:\Users\Harsh\Downloads\data_science01\dl_projects\Age_data_work\Trained cnn model\age_prediction_wiki_model.h5")

# Function to preprocess the image and predict the age
def predict_age(img_path):
    img = cv2.imread(img_path)
    if img is not None:
        # Resize the image to 128x128
        img_resized = cv2.resize(img, (128, 128))

        # Normalize the image to [0, 1]
        img_normalized = img_resized / 255.0

        # Expand dimensions to match model input
        img_expanded = np.expand_dims(img_normalized, axis=0)

        # Predict the age
        prediction = model.predict(img_expanded)
        predicted_age = prediction[0][0]
        return predicted_age
    else:
        return None

# Function to open a file dialog and choose an image
def open_image():
    file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg *.jpeg *.png")])
    if file_path:
        img = Image.open(file_path)
        img.thumbnail((250, 250))
        img_tk = ImageTk.PhotoImage(img)
        label_img.config(image=img_tk)
        label_img.image = img_tk

        # Predict age
        age = predict_age(file_path)
        if age is not None:
            label_result.config(text=f"Predicted Age: {int(age)}")
        else:
            label_result.config(text="Error: Could not process the image")

# Create the main window
root = tk.Tk()
root.title("Age Prediction")

# Create and place the widgets
button_open = tk.Button(root, text="Open Image", command=open_image)
button_open.pack()

label_img = Label(root)
label_img.pack()

label_result = Label(root, text="Predicted Age: ", font=("Helvetica", 16))
label_result.pack()

# Run the GUI event loop
root.mainloop()
